In [1]:
import sys
import tensorflow as tf
sys.path.append("/home/ubuntu/bob/Tensorbob")
sys.path.append("/home/ubuntu/bob/models/research/slim")
import tensorbob as bob
import pandas as pd
import numpy as np
import os
import tensorflow.contrib.slim as slim

In [2]:
DOG_BREED_ROOT = "/home/ubuntu/data/kaggle/dogbreed/"
LABELS_FILE = os.path.join(DOG_BREED_ROOT, 'labels.csv')
TRAIN_DIR = os.path.join(DOG_BREED_ROOT, 'train')
TEST_DIR = os.path.join(DOG_BREED_ROOT, 'test')
SAMPLE_FILE = os.path.join(DOG_BREED_ROOT, 'sample_submission.csv')

In [3]:
df = pd.read_csv(LABELS_FILE)
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
n = len(df)
breed = set(df['breed'])
n_class = len(breed)
class_to_num = dict(zip(breed, range(n_class)))
num_to_class = dict(zip(range(n_class), breed))

In [5]:
TRAIN_IMAGES = [os.path.join(TRAIN_DIR, df['id'][i]+'.jpg') for i in range(n)]
LABELS = [class_to_num[df['breed'][i]]  for i in range(n)]

In [6]:
num_classes = 120
log_every_n_step = 10
number_of_steps = 4400
batch_size = 64
learning_rate = 0.001
image_size = 224

kwargs = {'image_width': image_size,
          'image_height': image_size, 
          'norm_fn': bob.data.norm_imagenet, 
          'crop_width': image_size, 
          'crop_height': image_size, 
          'central_crop_flag': False, 
          'random_flip_horizontal_flag': True, 
          'random_flip_vertical_flag': True} 

def get_dog_breed_dataset(images=TRAIN_IMAGES, labels=LABELS, batch_size=32, **kwargs):
    images_config = bob.data.get_image_by_path_dataset_config(images, **kwargs)
    labels_config = bob.data.get_labels_dataset_config(labels)
    dataset_configs = [images_config, labels_config]
    return bob.data.BaseDataset(dataset_configs, batch_size, repeat=True)

train_dataset = get_dog_breed_dataset(batch_size=batch_size, **kwargs)
train_dataset.size

10222

In [7]:
import nets.vgg as vgg

ph_images = tf.placeholder(tf.float32, [None, image_size, image_size, 3])
ph_labels = tf.placeholder(tf.int32, [None])
ph_is_training = tf.placeholder(tf.bool)
with bob.arg_scope(vgg.vgg_arg_scope()):
    logits, end_points = vgg.vgg_16(ph_images, num_classes, global_pool=True, is_training=ph_is_training)

In [8]:
bob.losses.sparse_softmax_cross_entropy(ph_labels, logits)
total_loss = bob.losses.get_total_loss()
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
train_op = bob.training.create_train_op(total_loss, optimizer, global_step=bob.variables.get_or_create_global_step())

In [9]:
def feed_fn(sess):
    cur_images, cur_labels = train_dataset.get_next_batch(sess)
    result_dict = {ph_images: cur_images, ph_labels: cur_labels, ph_is_training: True}
    return result_dict
feed_fn_hook = bob.training_utils.FeedFnWithSessionHook(feed_fn)

In [10]:
metric_accuracy, accuracy_update_op = tf.metrics.accuracy(ph_labels, tf.argmax(logits, 1))

LOGS_DIR = "/home/ubuntu/bob/Tensorbob/examples/dogbreed/logs"
metrics_dict = {total_loss: None, metric_accuracy: accuracy_update_op}
metrics_epoch_hook = bob.training_utils.MetricsEpochMeanHook(metrics_dict, 20, LOGS_DIR)
train_op = metrics_epoch_hook.wrap_train_op(train_op)

INFO:tensorflow:Summary name metrics/mean/total_loss:0 is illegal; using metrics/mean/total_loss_0 instead.
INFO:tensorflow:Summary name metrics/mean/accuracy/value:0 is illegal; using metrics/mean/accuracy/value_0 instead.


In [11]:
logging_tensors = {"total_loss": total_loss}
for metric in metrics_epoch_hook.metrics_sums:
    logging_tensors[metric.name] = metric
total_loss_logging_hook = tf.train.LoggingTensorHook(logging_tensors, every_n_iter=10)

In [12]:
bob.training.train(train_op, LOGS_DIR, hooks=[feed_fn_hook, metrics_epoch_hook], max_steps=80)

epoch 1.0: 
metrics/mean/total_loss:0 7.0732
metrics/mean/accuracy/value:0 0.0096
epoch 2.0: 
metrics/mean/total_loss:0 7.0660
metrics/mean/accuracy/value:0 0.0083
epoch 3.0: 
metrics/mean/total_loss:0 7.0668
metrics/mean/accuracy/value:0 0.0075
epoch 4.0: 
metrics/mean/total_loss:0 7.0610
metrics/mean/accuracy/value:0 0.0085
